In [16]:

!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/survey_results_public.csv -P ./down


--2023-07-05 02:30:12--  https://s3-geospatial.s3-us-west-2.amazonaws.com/
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.92.145.34, 52.92.152.106, 52.218.242.249, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.92.145.34|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2023-07-05 02:30:13 ERROR 403: Forbidden.



In [63]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf()
conf.set('spark.app.name','pyspark_stack')
conf.set('spark.master','local[*]')

spark = SparkSession.builder\
    .config(conf=conf)\
    .getOrCreate()

In [23]:
df = spark.read.csv('down/survey_results_public.csv',header=True).select('ResponseId','LanguageHaveWorkedWith','LanguageWantToWorkWith')
df.show()

+----------+----------------------+----------------------+
|ResponseId|LanguageHaveWorkedWith|LanguageWantToWorkWith|
+----------+----------------------+----------------------+
|         1|  C++;HTML/CSS;Java...|                 Swift|
|         2|     JavaScript;Python|                    NA|
|         3|  Assembly;C;Python...|     Julia;Python;Rust|
|         4|  JavaScript;TypeSc...|  JavaScript;TypeSc...|
|         5|  Bash/Shell;HTML/C...|  Bash/Shell;HTML/C...|
|         6|  C;C#;C++;HTML/CSS...|  C#;C++;Go;HTML/CS...|
|         7|   HTML/CSS;JavaScript|  HTML/CSS;JavaScri...|
|         8|  HTML/CSS;JavaScri...|                  Ruby|
|         9|   HTML/CSS;JavaScript|   HTML/CSS;JavaScript|
|        10|            C++;Python|        Haskell;Python|
|        11|         C#;TypeScript|                    C#|
|        12|  Bash/Shell;HTML/C...|  C++;Clojure;JavaS...|
|        13|  C;C++;Java;Perl;Ruby|                  Rust|
|        14|  C;C++;JavaScript;...|                    N

In [24]:
import pyspark.sql.functions as F

df2 = df.withColumn(
    'language_have',
    F.split(F.trim(F.col("LanguageHaveWorkedWith")),";")
)
df2.show()

+----------+----------------------+----------------------+--------------------+
|ResponseId|LanguageHaveWorkedWith|LanguageWantToWorkWith|       language_have|
+----------+----------------------+----------------------+--------------------+
|         1|  C++;HTML/CSS;Java...|                 Swift|[C++, HTML/CSS, J...|
|         2|     JavaScript;Python|                    NA|[JavaScript, Python]|
|         3|  Assembly;C;Python...|     Julia;Python;Rust|[Assembly, C, Pyt...|
|         4|  JavaScript;TypeSc...|  JavaScript;TypeSc...|[JavaScript, Type...|
|         5|  Bash/Shell;HTML/C...|  Bash/Shell;HTML/C...|[Bash/Shell, HTML...|
|         6|  C;C#;C++;HTML/CSS...|  C#;C++;Go;HTML/CS...|[C, C#, C++, HTML...|
|         7|   HTML/CSS;JavaScript|  HTML/CSS;JavaScri...|[HTML/CSS, JavaSc...|
|         8|  HTML/CSS;JavaScri...|                  Ruby|[HTML/CSS, JavaSc...|
|         9|   HTML/CSS;JavaScript|   HTML/CSS;JavaScript|[HTML/CSS, JavaSc...|
|        10|            C++;Python|     

In [27]:
df3 = df2.withColumn(
    'language_want',
    F.split(F.trim(F.col("LanguageWantToWorkWith")),";")
)
df3.show()

+----------+----------------------+----------------------+--------------------+--------------------+
|ResponseId|LanguageHaveWorkedWith|LanguageWantToWorkWith|       language_have|       language_want|
+----------+----------------------+----------------------+--------------------+--------------------+
|         1|  C++;HTML/CSS;Java...|                 Swift|[C++, HTML/CSS, J...|             [Swift]|
|         2|     JavaScript;Python|                    NA|[JavaScript, Python]|                [NA]|
|         3|  Assembly;C;Python...|     Julia;Python;Rust|[Assembly, C, Pyt...|[Julia, Python, R...|
|         4|  JavaScript;TypeSc...|  JavaScript;TypeSc...|[JavaScript, Type...|[JavaScript, Type...|
|         5|  Bash/Shell;HTML/C...|  Bash/Shell;HTML/C...|[Bash/Shell, HTML...|[Bash/Shell, HTML...|
|         6|  C;C#;C++;HTML/CSS...|  C#;C++;Go;HTML/CS...|[C, C#, C++, HTML...|[C#, C++, Go, HTM...|
|         7|   HTML/CSS;JavaScript|  HTML/CSS;JavaScri...|[HTML/CSS, JavaSc...|[HTML/CSS, J

In [31]:
df_language_have = df3.select(
    df3.ResponseId,
    F.explode(df3.language_have).alias('language_have')
)
df_language_have.show()

+----------+-------------+
|ResponseId|language_have|
+----------+-------------+
|         1|          C++|
|         1|     HTML/CSS|
|         1|   JavaScript|
|         1|  Objective-C|
|         1|          PHP|
|         1|        Swift|
|         2|   JavaScript|
|         2|       Python|
|         3|     Assembly|
|         3|            C|
|         3|       Python|
|         3|            R|
|         3|         Rust|
|         4|   JavaScript|
|         4|   TypeScript|
|         5|   Bash/Shell|
|         5|     HTML/CSS|
|         5|       Python|
|         5|          SQL|
|         6|            C|
+----------+-------------+
only showing top 20 rows



In [33]:
df_language_have.groupby('language_have').count().show()

+-------------+-----+
|language_have|count|
+-------------+-----+
|           C#|22984|
|          VBA| 3847|
|         Rust| 5799|
|   Bash/Shell|22385|
|   JavaScript|53587|
|           NA| 1082|
|         Perl| 2028|
|       Erlang|  651|
|       Matlab| 3846|
|      Crystal|  466|
|          C++|20057|
|           F#|  804|
|       Groovy| 2479|
|         Dart| 4965|
|          APL|  536|
|       Delphi| 1731|
|        Julia| 1068|
|  Objective-C| 2310|
|            C|17329|
|        Swift| 4204|
+-------------+-----+
only showing top 20 rows



# Sort & orderBY
# ascending & descending


In [40]:
df_language_have.groupby('language_have').count().sort(F.desc("count")).show(100)

+-------------+-----+
|language_have|count|
+-------------+-----+
|   JavaScript|53587|
|     HTML/CSS|46259|
|       Python|39792|
|          SQL|38835|
|         Java|29162|
|      Node.js|27975|
|   TypeScript|24909|
|           C#|22984|
|   Bash/Shell|22385|
|          C++|20057|
|          PHP|18130|
|            C|17329|
|   PowerShell| 8871|
|           Go| 7879|
|       Kotlin| 6866|
|         Rust| 5799|
|         Ruby| 5569|
|         Dart| 4965|
|     Assembly| 4632|
|        Swift| 4204|
|            R| 4185|
|          VBA| 3847|
|       Matlab| 3846|
|       Groovy| 2479|
|  Objective-C| 2310|
|        Scala| 2148|
|         Perl| 2028|
|      Haskell| 1749|
|       Delphi| 1731|
|      Clojure| 1552|
|       Elixir| 1438|
|         LISP| 1096|
|           NA| 1082|
|        Julia| 1068|
|           F#|  804|
|       Erlang|  651|
|          APL|  536|
|      Crystal|  466|
|        COBOL|  437|
+-------------+-----+



In [52]:
df_50 = df_language_have.groupby('language_have').count().orderBy('count', ascending=False)


In [53]:
df_50.write.mode('overwrite').csv('lang_50.csv')

In [54]:
!ls -tl

total 1728
-rw-r--r--@ 1 giho.han  staff   17349 Jul  5 02:50 stackoverflow_survey.ipynb
drwxr-xr-x@ 6 giho.han  staff     192 Jul  5 02:50 lang_50.csv
drwxr-xr-x@ 3 giho.han  staff      96 Jul  5 02:29 down
-rw-r--r--@ 1 giho.han  staff   18429 Jul  5 02:22 String_parse.ipynb
drwxr-xr-x@ 6 giho.han  staff     192 Jul  5 02:20 extracted.json
drwxr-xr-x@ 6 giho.han  staff     192 Jul  5 02:20 extract2.csv
drwxr-xr-x@ 6 giho.han  staff     192 Jul  5 02:16 extract.csv
-rw-r--r--@ 1 giho.han  staff    8893 Jul  5 01:43 colab_main2.ipynb
drwxr-xr-x@ 2 giho.han  staff      64 Jul  5 01:43 spark-warehouse
-rw-r--r--@ 1 giho.han  staff   26077 Jul  5 01:18 colab_main1.ipynb
-rw-r--r--@ 1 giho.han  staff      10 Jul  5 00:10 README.md
-rw-r--r--@ 1 giho.han  staff     997 Jul  5 00:06 name_gender.csv
-rw-r--r--@ 1 giho.han  staff     519 Jul  5 00:06 main.py
-rw-r--r--@ 1 giho.han  staff  136855 Jul  5 00:06 customer-orders.csv
-rw-r--r--@ 1 giho.han  staff   62728 Jul  5 00:06 1800.csv
drwxr-

In [57]:
df_want = df3.select(
    df3.ResponseId,
    F.explode(df3.language_want).alias('language_want')
)

In [58]:
df_want.show()

+----------+-------------+
|ResponseId|language_want|
+----------+-------------+
|         1|        Swift|
|         2|           NA|
|         3|        Julia|
|         3|       Python|
|         3|         Rust|
|         4|   JavaScript|
|         4|   TypeScript|
|         5|   Bash/Shell|
|         5|     HTML/CSS|
|         5|       Python|
|         5|          SQL|
|         6|           C#|
|         6|          C++|
|         6|           Go|
|         6|     HTML/CSS|
|         6|         Java|
|         6|   JavaScript|
|         6|      Node.js|
|         6|  Objective-C|
|         6|         Perl|
+----------+-------------+
only showing top 20 rows



In [61]:
fin_want_df = df_want.groupby('language_want').count().orderBy('count',ascending=False)

In [62]:
fin_want_df.write.mode('overwrite').csv('fin_want_df.csv')